In [4]:
import os

In [5]:
%pwd

'c:\\Users\\rohit\\Desktop\\Files\\Projects\\ML-Project-with-MLflow\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\rohit\\Desktop\\Files\\Projects\\ML-Project-with-MLflow'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories

In [10]:
%pwd

'c:\\Users\\rohit\\Desktop\\Files\\Projects\\ML-Project-with-MLflow'

In [11]:
from mlFlowProject.constants import CONFIG_FILE_PATH
print(CONFIG_FILE_PATH)

config\config.yaml


In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

    

In [13]:
import os
import urllib.request as request
import zipfile
from mlFlowProject import logger
from mlFlowProject.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-26 14:42:41,082: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-26 14:42:41,085: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-26 14:42:41,087: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-26 14:42:41,089: INFO: common: created directory at: artifacts]
[2025-11-26 14:42:41,090: INFO: common: created directory at: artifacts/data_ingestion]
[2025-11-26 14:42:41,092: INFO: 4076605423: File already exists of size: ~ 23 KB]
